Script de select con funciones agregadas Max. Min y Avg de datos MySQL en cluster multidomain

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import sqlalchemy
import time
import matplotlib.pyplot as plt
import psutil
import random

from mysql.connector.cursor import MySQLCursor

In [2]:
#Los resultados de medidas de tiempo en carga por dominios se almacenan en estos objetos.
#Se itera durante 100 iteraciones para sacar medias
#repeticiones
repeats = 100

In [3]:
#Ficheros de salida
resultados_etl_agg = '../Results/MySQL/MySQLFuncionesAgregadas_test_{}.csv'

In [4]:
def save_results_to_csv(results,file):
    #Guardamos los resultados en csv
    from datetime import datetime
    
    csv_df = pd.DataFrame(results, columns=['Registros', 'Tiempo', 'CPU','Memoria'])
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S")
    csv_df.to_csv(file.format(str(dia)))

In [5]:
conn = mysql.connector.connect(host='localhost',
                                    port=6446,
                                    database='CustomerProfileDomainSchema',
                                    user='root',                                       
                                    password='mysql')
if conn.is_connected():
        conn.autocommit = False #Apenas se observa diferencia de rendimiento
        print('Connected to MySQL database')
    
cursor = conn.cursor()  
    
def connection_open():
   
    #InnoDB Cluster
    conn = mysql.connector.connect(host='localhost',
                                    port=6446,
                                    database='CustomerProfileDomainSchema',
                                    user='root',                                       
                                    password='mysql')


    if conn.is_connected():
        conn.autocommit = False #Apenas se observa diferencia de rendimiento
        print('Connected to MySQL database')
    
    cursor = conn.cursor()  

Connected to MySQL database


In [6]:
cursor = conn.cursor(buffered=True)
cursor.execute('set global group_replication_transaction_size_limit = 300000000;')
cursor.close()

True

In [7]:
#Obtenemos los posibles valores de pais. Se iterará por ellos para cambiar en bucle los registros
cursor = conn.cursor(buffered=True)
cursor.execute("""SELECT * FROM PositionKeepingDomainSchema.Currency""")
country_list= []
for code in cursor.fetchall():
    country_list.append(str(code[1]))
country_list

['IND', 'GBP', 'EUR', 'USA']

# Select test multidomain

In [8]:
time_inicial = 0
time_final = 0
registers = []
   
for iteracion in range(0,repeats): 
    select_query = """SELECT AVG(Amount),MIN(Amount),MAX(Amount) FROM PositionKeepingDomainSchema.Amount am
INNER JOIN PositionKeepingDomainSchema.PositionKeeping pk ON pk.AmountId = am.AmountId
INNER JOIN CurrentAccountDomainSchema.CurrentAccount ca ON ca.AccountId = pk.AccountId
INNER JOIN CustomerProfileDomainSchema.CustomerProfile cp ON cp.PartyId = ca.PartyId
INNER JOIN CurrentAccountDomainSchema.AccountInfo ai ON ai.AccountId = ca.AccountId
INNER JOIN PositionKeepingDomainSchema.Currency cr ON cr.CurrencyId = am.CurrencyId
WHERE cr.Code = '{}'
AND cp.AccountRole = 'UK.OBIE.Principal'
AND ai.SchemeName LIKE 'UK.%'""".format(random.choice(country_list))
    cursor = conn.cursor(buffered=True)
    time_inicial = time.time()
    cursor.execute(select_query)
    time_final = time.time() 
    used_cpu = psutil.cpu_percent()
    mem_used = psutil.virtual_memory().percent
    # Tupla con numero de registros, tiempo parcial de la transacción y tiempo acumulado de trxs
    total_time = round(time_final - time_inicial,3)
    registers.append((iteracion + 1, total_time ,used_cpu, mem_used))
    #print((iteracion + 1 , total_time ,used_cpu, mem_used))
    cursor.close()
conn.close()

In [9]:
#Guardamos los resultados Customer Profile
save_results_to_csv(registers,resultados_etl_agg)

In [10]:
registers

[(1, 0.627, 19.6, 49.2),
 (2, 0.608, 18.4, 49.2),
 (3, 0.605, 18.8, 49.2),
 (4, 0.599, 17.4, 49.2),
 (5, 0.618, 27.0, 49.2),
 (6, 0.591, 17.8, 49.2),
 (7, 0.612, 18.2, 49.2),
 (8, 0.617, 18.4, 49.2),
 (9, 0.65, 29.3, 49.2),
 (10, 0.619, 20.1, 49.2),
 (11, 0.621, 19.3, 49.2),
 (12, 0.615, 19.7, 49.2),
 (13, 0.618, 21.5, 49.2),
 (14, 0.609, 19.5, 49.2),
 (15, 0.609, 19.7, 49.2),
 (16, 0.626, 19.1, 49.2),
 (17, 0.61, 17.3, 49.2),
 (18, 0.614, 17.7, 49.2),
 (19, 0.611, 18.0, 49.2),
 (20, 0.608, 17.6, 49.2),
 (21, 0.6, 17.3, 49.2),
 (22, 0.599, 18.0, 49.2),
 (23, 0.603, 17.7, 49.2),
 (24, 0.6, 18.3, 49.2),
 (25, 0.579, 17.8, 49.2),
 (26, 0.614, 17.8, 49.2),
 (27, 0.61, 17.3, 49.2),
 (28, 0.615, 18.1, 49.2),
 (29, 0.612, 18.2, 49.2),
 (30, 0.609, 18.0, 49.2),
 (31, 0.605, 17.4, 49.2),
 (32, 0.6, 17.8, 49.2),
 (33, 0.601, 18.1, 49.2),
 (34, 0.593, 17.7, 49.2),
 (35, 0.634, 24.1, 49.3),
 (36, 0.62, 17.5, 49.3),
 (37, 0.611, 17.5, 49.3),
 (38, 0.608, 17.3, 49.3),
 (39, 0.609, 18.3, 49.3),
 (40,

In [11]:
conn.close()
print('Conexion cerrada')

Conexion cerrada
